In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/mba-admission/Admission.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df["Decision"].unique()

In [ ]:
plt.figure(figsize=(12,6))
sns.kdeplot(data=df[df["Decision"]=="admit"],x="GMAT",shade=True,label="Admitted")
sns.kdeplot(data=df[df["Decision"]=="border"],x="GMAT",color="green",shade=True,label="Border")
sns.kdeplot(data=df[df["Decision"]=="notadmit"],x="GMAT",color="orange",shade=True,label="Declined")

plt.title("GMAT Distributions")
plt.legend()

In [ ]:
plt.figure(figsize=(12,6))
sns.kdeplot(data=df[df["Decision"]=="admit"],x="GPA",shade=True,label="Admitted")
sns.kdeplot(data=df[df["Decision"]=="border"],x="GPA",color="green",shade=True,label="Border")
sns.kdeplot(data=df[df["Decision"]=="notadmit"],x="GPA",color="orange",shade=True,label="Declined")

plt.title("GPA Distributions")
plt.legend()

In [ ]:
sns.jointplot(data=df,x="GPA",y="GMAT",hue="Decision")

In [ ]:
# Explained variance between features (R^2)
ndf = df.copy()

ndf["Decision"] = ndf["Decision"].astype("category")
ndf["Decision"] = ndf["Decision"].cat.codes

sns.heatmap(ndf.corr()**2,annot=True,cmap="viridis")

In [ ]:
ndf.head()

In [ ]:
X = ndf.drop("Decision",axis=1)

y = ndf["Decision"]

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

rfr = RandomForestClassifier()
knn = KNeighborsClassifier()
logmod = LogisticRegression()

rfr.fit(X_train, y_train)
knn.fit(X_train, y_train)
logmod.fit(X_train, y_train)

print("Random Forest Regressor Score: {:.2f}".format(rfr.score(X_test, y_test)))
print("K Nearest Neighbors Score: {:.2f}".format(knn.score(X_test, y_test)))
print("Logistic Regression Score: {:.2f}".format(logmod.score(X_test, y_test)))

In [ ]:
error = []
for k in range(1,40):
    
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    p = knn.predict(X_test)
    
    error.append(np.mean(y_test != p))

In [ ]:
# Visualizing optimal k value
plt.plot(range(1,40),error,ls="--",
        marker="X",markerfacecolor="red")

In [ ]:
# Mean score of 50 splits and fits
score = []
for i in range(1,50):
    
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25) 

    knn = KNeighborsClassifier(n_neighbors=3)

    knn.fit(X_train, y_train)
    
    score.append(knn.score(X_test,y_test))
    
np.array(score).mean()